# Deep Learning Project - STL-10 Dataset Classification

## Project Requirements:
1. Dataset: STL-10 (Similar to CIFAR-10 but different dataset)
2. Task: Classification
3. EDA (Exploratory Data Analysis)
4. Data Preprocessing
5. Deep Learning Model (ResNet-32)
6. Validation Loss Reduction
7. Learning Rate Scheduling
8. Early Stopping
9. Data Augmentation
10. Classification Report & Accuracy
11. RandomSearchCV for Hyperparameter Tuning


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import callbacks
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)



In [ ]:
# Print versions for reference
print(f"NumPy: {np.__version__}")
print(f"TensorFlow: {tf.__version__}")
print(f"Keras: {keras.__version__}")


## 1. Dataset Loading - STL-10

STL-10 dataset contains 13,000 color images in 10 classes (airplane, bird, car, cat, deer, dog, horse, monkey, ship, truck). 
Images are 96x96 pixels, similar to CIFAR-10 but a different dataset.


In [ ]:
# Load STL-10 dataset - Simple and Short (No Loops!)
print("Loading STL-10 dataset...")

# Load and combine all data at once
ds_train, ds_test = tfds.load('stl10', split=['train', 'test'], as_supervised=True, batch_size=-1)
X_all = np.concatenate([ds_train[0].numpy(), ds_test[0].numpy()], axis=0)
y_all = np.concatenate([ds_train[1].numpy().flatten(), ds_test[1].numpy().flatten()], axis=0)

# Split: 70% train, 15% val, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X_all, y_all, test_size=0.3, random_state=42, stratify=y_all)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Resize first
X_train = tf.image.resize(X_train.astype('float32'), [32, 32]).numpy()
X_val = tf.image.resize(X_val.astype('float32'), [32, 32]).numpy()
X_test = tf.image.resize(X_test.astype('float32'), [32, 32]).numpy()

# Normalize after resize
X_train = X_train / 255.0
X_val = X_val / 255.0
X_test = X_test / 255.0

print(f"Train: {len(X_train)} (70%) | Val: {len(X_val)} (15%) | Test: {len(X_test)} (15%)")


## 2. Task Identification: Classification

This is a **Multi-class Classification** problem with 10 classes:
- 0: airplane
- 1: bird
- 2: car
- 3: cat
- 4: deer
- 5: dog
- 6: horse
- 7: monkey
- 8: ship
- 9: truck


## 3. Exploratory Data Analysis (EDA)


In [ ]:
### 3.1 Sample Images Visualization


In [ ]:
# Step 1: Display sample images from each class
class_names = ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']

fig, axes = plt.subplots(2, 5, figsize=(15, 6))
fig.suptitle('Sample Images from Each Class', fontsize=16)

# Get first image of each class - Direct access (No loops!)
idx0 = np.where(y_train == 0)[0][0]; idx1 = np.where(y_train == 1)[0][0]
idx2 = np.where(y_train == 2)[0][0]; idx3 = np.where(y_train == 3)[0][0]
idx4 = np.where(y_train == 4)[0][0]; idx5 = np.where(y_train == 5)[0][0]
idx6 = np.where(y_train == 6)[0][0]; idx7 = np.where(y_train == 7)[0][0]
idx8 = np.where(y_train == 8)[0][0]; idx9 = np.where(y_train == 9)[0][0]

# Display images with class names - Direct (No loops!)
axes[0, 0].imshow(X_train[idx0]); axes[0, 0].set_title(f'Class 0: {class_names[0]}'); axes[0, 0].axis('off')
axes[0, 1].imshow(X_train[idx1]); axes[0, 1].set_title(f'Class 1: {class_names[1]}'); axes[0, 1].axis('off')
axes[0, 2].imshow(X_train[idx2]); axes[0, 2].set_title(f'Class 2: {class_names[2]}'); axes[0, 2].axis('off')
axes[0, 3].imshow(X_train[idx3]); axes[0, 3].set_title(f'Class 3: {class_names[3]}'); axes[0, 3].axis('off')
axes[0, 4].imshow(X_train[idx4]); axes[0, 4].set_title(f'Class 4: {class_names[4]}'); axes[0, 4].axis('off')
axes[1, 0].imshow(X_train[idx5]); axes[1, 0].set_title(f'Class 5: {class_names[5]}'); axes[1, 0].axis('off')
axes[1, 1].imshow(X_train[idx6]); axes[1, 1].set_title(f'Class 6: {class_names[6]}'); axes[1, 1].axis('off')
axes[1, 2].imshow(X_train[idx7]); axes[1, 2].set_title(f'Class 7: {class_names[7]}'); axes[1, 2].axis('off')
axes[1, 3].imshow(X_train[idx8]); axes[1, 3].set_title(f'Class 8: {class_names[8]}'); axes[1, 3].axis('off')
axes[1, 4].imshow(X_train[idx9]); axes[1, 4].set_title(f'Class 9: {class_names[9]}'); axes[1, 4].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Step 2: Visualize class distribution for all datasets
class_names = ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']

plt.figure(figsize=(16, 5))

plt.subplot(1, 3, 1)
class_counts = np.bincount(y_train)
plt.bar(range(10), class_counts, color='skyblue', edgecolor='black')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Training Set (70%)')
plt.xticks(range(10), class_names, rotation=45, ha='right')

plt.subplot(1, 3, 2)
class_counts_val = np.bincount(y_val)
plt.bar(range(10), class_counts_val, color='lightgreen', edgecolor='black')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Validation Set (15%)')
plt.xticks(range(10), class_names, rotation=45, ha='right')

plt.subplot(1, 3, 3)
class_counts_test = np.bincount(y_test)
plt.bar(range(10), class_counts_test, color='lightcoral', edgecolor='black')
plt.xlabel('Class')
plt.ylabel('Count')
plt.title('Test Set (15%)')
plt.xticks(range(10), class_names, rotation=45, ha='right')

plt.tight_layout()
plt.show()


In [ ]:
# Step 3: Print class distribution summary
print("=== Class Distribution Summary ===")
print("Training set:")
print(f"Class 0 ({class_names[0]}): {class_counts[0]} samples")
print(f"Class 1 ({class_names[1]}): {class_counts[1]} samples")
print(f"Class 2 ({class_names[2]}): {class_counts[2]} samples")
print(f"Class 3 ({class_names[3]}): {class_counts[3]} samples")
print(f"Class 4 ({class_names[4]}): {class_counts[4]} samples")
print(f"Class 5 ({class_names[5]}): {class_counts[5]} samples")
print(f"Class 6 ({class_names[6]}): {class_counts[6]} samples")
print(f"Class 7 ({class_names[7]}): {class_counts[7]} samples")
print(f"Class 8 ({class_names[8]}): {class_counts[8]} samples")
print(f"Class 9 ({class_names[9]}): {class_counts[9]} samples")


### 3.3 Dataset Basic Information


In [ ]:
# Step 4: Dataset basic information
print("=== Dataset Basic Information ===")
print(f"Total samples: {len(X_all)}")
print(f"Training samples: {len(X_train)} ({len(X_train)/len(X_all)*100:.1f}%)")
print(f"Validation samples: {len(X_val)} ({len(X_val)/len(X_all)*100:.1f}%)")
print(f"Test samples: {len(X_test)} ({len(X_test)/len(X_all)*100:.1f}%)")
print(f"\nImage shape: {X_train[0].shape}")
print(f"Image dimensions: {X_train.shape[1]}x{X_train.shape[2]}x{X_train.shape[3]}")
print(f"Data type: {X_train.dtype}")
print(f"\nUnique classes: {len(np.unique(y_train))}")
print(f"Class labels: {np.unique(y_train)}")


### 3.4 Dataset Statistics


In [ ]:
# Step 5: Dataset statistics
print("=== Dataset Statistics ===")
print(f"Pixel value range: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"Mean pixel value: {X_train.mean():.4f}")
print(f"Std pixel value: {X_train.std():.4f}")
print(f"Median pixel value: {np.median(X_train):.4f}")


### 3.5 Missing Values Check


In [ ]:
# Step 6: Check for missing values
print("=== Missing Values Check ===")
print(f"Missing values in train: {np.isnan(X_train).sum()}")
print(f"Missing values in val: {np.isnan(X_val).sum()}")
print(f"Missing values in test: {np.isnan(X_test).sum()}")
print(f"\nMissing labels in train: {np.isnan(y_train).sum()}")
print(f"Missing labels in val: {np.isnan(y_val).sum()}")
print(f"Missing labels in test: {np.isnan(y_test).sum()}")


## 4. Data Preprocessing


In [ ]:
# Data Preprocessing - Simple and Easy
print("=== Data Preprocessing ===")

# Images: Already normalized (Min-Max: divided by 255.0) - Done in Cell 4
# Labels: Convert to One-Hot Encoding
y_train_categorical = keras.utils.to_categorical(y_train, 10)
y_val_categorical = keras.utils.to_categorical(y_val, 10)
y_test_categorical = keras.utils.to_categorical(y_test, 10)

print(f"Images normalized: [{X_train.min():.3f}, {X_train.max():.3f}]")
print(f"Labels one-hot encoded: Train {y_train_categorical.shape} | Val {y_val_categorical.shape} | Test {y_test_categorical.shape}")
print("=== Preprocessing Complete ===")


## 5. ResNet-32 Model Architecture

This section includes:
- Data Augmentation
- ResNet-32 Model Building
- Learning Rate Scheduling
- Early Stopping
- Model Training


### 5.1 Data Augmentation


In [ ]:
# Data Augmentation - Simple
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])
print("Data Augmentation: Random Flip, Rotation, Zoom")


### 5.2 ResNet-32 Model


In [ ]:
# ResNet-32 Model - Simple and Effective
def residual_block(x, filters):
    shortcut = x
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(filters, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    if shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, 1)(shortcut)
    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

# Build ResNet-32
inputs = layers.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = layers.Conv2D(32, 3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

# ResNet blocks (32 layers total)
x = residual_block(x, 32)
x = residual_block(x, 32)
x = residual_block(x, 32)
x = layers.MaxPooling2D(2)(x)

x = residual_block(x, 64)
x = residual_block(x, 64)
x = residual_block(x, 64)
x = layers.MaxPooling2D(2)(x)

x = residual_block(x, 128)
x = residual_block(x, 128)
x = residual_block(x, 128)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print("ResNet-32 Model Created")
model.summary()


### 5.3 Callbacks (Learning Rate Scheduling + Early Stopping)


In [ ]:
# Learning Rate Scheduling + Early Stopping
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
callbacks_list = [lr_scheduler, early_stopping]
print("Callbacks: Learning Rate Scheduling + Early Stopping")


### 5.4 Model Training


In [ ]:
# Train Model
print("Training ResNet-32...")
history = model.fit(
    X_train, y_train_categorical,
    validation_data=(X_val, y_val_categorical),
    epochs=50,
    batch_size=64,
    callbacks=callbacks_list,
    verbose=1
)
print("Training Complete!")


## 6. Model Evaluation


### 6.1 Training History Visualization


In [ ]:
# Plot Training History
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()

plt.tight_layout()
plt.show()


### 6.2 Test Set Evaluation


In [ ]:
# Evaluate on Test Set
test_loss, test_accuracy = model.evaluate(X_test, y_test_categorical, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")


### 6.3 Classification Report


In [ ]:
# Classification Report
y_pred = model.predict(X_test, verbose=0)
y_pred_classes = np.argmax(y_pred, axis=1)

class_names = ['airplane', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck']
print("Classification Report:")
print(classification_report(y_test, y_pred_classes, target_names=class_names))


### 6.4 Confusion Matrix


In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


## 7. Hyperparameter Tuning (RandomSearchCV)


In [ ]:
# RandomSearchCV for Hyperparameter Tuning - Simple and Working
print("=== RandomSearchCV Hyperparameter Tuning ===")

# Define model creation function
def create_model_for_search(learning_rate=0.001, dropout_rate=0.5, dense_units=128):
    def residual_block(x, filters):
        shortcut = x
        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)
        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        if shortcut.shape[-1] != filters:
            shortcut = layers.Conv2D(filters, 1)(shortcut)
        x = layers.Add()([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    
    inputs = layers.Input(shape=(32, 32, 3))
    x = data_augmentation(inputs)
    x = layers.Conv2D(32, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = residual_block(x, 32)
    x = residual_block(x, 32)
    x = residual_block(x, 32)
    x = layers.MaxPooling2D(2)(x)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = residual_block(x, 64)
    x = layers.MaxPooling2D(2)(x)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(10, activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
                  loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Parameter grid for RandomSearchCV
param_grid = {
    'learning_rate': [0.001, 0.0005, 0.0001],
    'dropout_rate': [0.3, 0.5, 0.7],
    'dense_units': [64, 128, 256]
}

# Use subset for faster search
X_train_subset = X_train[:2000]
y_train_subset = y_train_categorical[:2000]
X_val_subset = X_val[:500]
y_val_subset = y_val_categorical[:500]

# Manual RandomSearchCV (Simple approach)
print("Running RandomSearchCV (5 random combinations)...")
results = []

# Randomly sample 5 combinations
np.random.seed(42)
for i in range(5):
    lr = np.random.choice(param_grid['learning_rate'])
    dr = np.random.choice(param_grid['dropout_rate'])
    du = np.random.choice(param_grid['dense_units'])
    
    print(f"\nTrial {i+1}/5: LR={lr}, Dropout={dr}, Dense={du}")
    temp_model = create_model_for_search(learning_rate=lr, dropout_rate=dr, dense_units=du)
    
    temp_model.fit(X_train_subset, y_train_subset, 
                   validation_data=(X_val_subset, y_val_subset),
                   epochs=10, batch_size=64, verbose=0)
    
    score = temp_model.evaluate(X_val_subset, y_val_subset, verbose=0)[1]
    results.append({'learning_rate': lr, 'dropout_rate': dr, 'dense_units': du, 'score': score})
    print(f"  Validation Accuracy: {score:.4f}")

# Find best parameters
best_result = max(results, key=lambda x: x['score'])
print("\n=== RandomSearchCV Results ===")
print(f"Best Parameters: {best_result}")
print(f"\nAll Results:")
for i, r in enumerate(results):
    print(f"  {i+1}. LR={r['learning_rate']}, Dropout={r['dropout_rate']}, Dense={r['dense_units']} - Score: {r['score']:.4f}")


## 8. Project Summary


In [ ]:
# Final Project Summary
print("="*60)
print("DEEP LEARNING PROJECT - STL-10 CLASSIFICATION")
print("="*60)
print(f"\nDataset: STL-10")
print(f"Total Samples: {len(X_all)}")
print(f"Train: {len(X_train)} (70%) | Val: {len(X_val)} (15%) | Test: {len(X_test)} (15%)")
print(f"\nModel: ResNet-32")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"\nFeatures Used:")
print("  ✓ Data Augmentation (Flip, Rotation, Zoom)")
print("  ✓ Learning Rate Scheduling (ReduceLROnPlateau)")
print("  ✓ Early Stopping")
print("  ✓ Batch Normalization")
print("  ✓ Dropout Regularization")
print("="*60)
